In [16]:
import os
import numpy as np
import pandas
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import random

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
DATA_PATH = '/home/zhaoyu/deepLeaf/Data/'
PROJECT_PATH = '/home/zhaoyu/deepLeaf/'

In [4]:
IMAGE_RESIZE_SIZE = 64

In [5]:
# read the train and test data

train = pandas.read_csv(DATA_PATH + '/train.csv')
test = pandas.read_csv(DATA_PATH + '/test.csv')

# get the pre-extracted features
feat_train = train.copy()
feat_test = test.copy()
feat_train = feat_train.drop(['id', 'species'], axis=1)
feat_test = feat_test.drop(['id'], axis=1)
feat_train = StandardScaler().fit(feat_train).transform(feat_train)
feat_test = StandardScaler().fit(feat_test).transform(feat_test)

### Data Augmentation Method 1: Resize directly

In [6]:
# Data augmentation
# image_data contains ALL images from the images folder

# method 1: resize directly
image_data = {}
for img_file in os.listdir(DATA_PATH + '/images'):
    resized_img = imresize(imread(DATA_PATH + '/images/' + img_file), (IMAGE_RESIZE_SIZE,IMAGE_RESIZE_SIZE)).astype(np.float32)
    image_data[img_file.split(".")[0]] = resized_img

### Data Augmentation Method 2: Padding first, then resize (image will not be disturbed)

In [ ]:
# method 2: pad first, then resize
image_data = {}
for img_file in os.listdir(DATA_PATH + '/images'):
    img = imread(DATA_PATH + '/images/' + img_file)
    h, w = img.shape
    max_dim = max(h, w)
    padded_img = np.lib.pad(img, 
                     (((max_dim-h)//2, max_dim-h-(max_dim-h)//2), ((max_dim-w)//2, max_dim-w-(max_dim-w)//2)), 
                     'constant', constant_values=1)
    resized_img = imresize(padded_img, (IMAGE_RESIZE_SIZE, IMAGE_RESIZE_SIZE)).astype(np.float32)
    image_data[img_file.split(".")[0]] = resized_img

### Data Augmentation Method 3: rotation/flip

In [7]:
# codes is from https://www.kaggle.com/abhmul/leaf-classification/keras-convnet-lb-0-0052-w-visualization

from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator, array_to_img


In [8]:
print(train.shape)
print(feat_train.shape)
print(test.shape)
print(feat_test.shape)
print(len(image_data))

(990, 194)
(990, 192)
(594, 193)
(594, 192)
1584


In [ ]:
# for key, value in image_data.items() :
#     print (key, value)
#     exit()

In [ ]:
plt.imshow(image_data['2'], cmap='gray')
print(image_data['1'].shape)
plt.show()

In [9]:
# get labels
le = LabelEncoder()
le.fit(train.species)
# print(le.classes_)
# print(len(le.classes_))

LabelEncoder()

In [10]:
train_labels = le.transform(train.species)
# print(train_labels)
# print(len(train_labels))

In [11]:
# separate train and test from image_data
image_train = np.array([image_data[str(idx)] for idx in train.id])
image_test = np.array([image_data[str(idx)] for idx in test.id])
image_train = np.expand_dims(image_train, axis=4)
image_test = np.expand_dims(image_test, axis=4)
print(image_train.shape)
print(image_test.shape)

(990, 64, 64, 1)
(594, 64, 64, 1)


In [30]:
# split train into train and validation
# Use heavy data augmentation
# image_train (990, 64, 64, 1)
# train_labels_cat (990, 99)

train_numbers = int(len(image_train) * 0.9)
val_numbers = len(image_train) - train_numbers
train_indices = random.sample(range(0, len(image_train)), train_numbers)
val_indices = [x for x in range(0, len(image_train)) if x not in train_indices]

image_train_splitted = image_train[train_indices, :, :, :]
image_train_label_splitted = train_labels_cat[train_indices, :]
image_validation_splitted = image_train[val_indices, :, :, :]
image_validation_label_splitted = train_labels_cat[val_indices, :]

print('image_train_splitted shape', image_train_splitted.shape)
print('image_validation_splitted shape', image_validation_splitted.shape)

image_train_splitted shape (891, 64, 64, 1)
image_validation_splitted shape (99, 64, 64, 1)


In [31]:
# image generator for splitted train and validation
imgen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)
imgen.fit(image_train_splitted)
imgen_flow = imgen.flow(image_train_splitted, image_train_label_splitted, batch_size=16)

In [ ]:
# # randomly split original train into train and validation
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
# for train_index, val_index in sss.split(image_train, train_labels):
#     image_train_X, image_val_X = image_train[train_index], image_train[val_index]
#     image_train_Y, image_val_Y = train_labels[train_index], train_labels[val_index]
#     print(image_train_X.shape)
#     print(image_train_Y.shape)
#     print(image_val_X.shape)
#     print(image_val_Y.shape)
#     print("TRAIN:", train_index)
#     print("VAL:", val_index)
#     plt.imshow(image_train_X[0, :, :, 0], cmap='gray')
#     plt.show()
#     print(train_index[0])
#     print(image_train_Y[0])
#     print(le.inverse_transform([image_train_Y[0]]))

In [12]:
# Convert class vectors to binary class matrices (one-hot encoding)
num_classes = 99
train_labels_cat = keras.utils.to_categorical(train_labels, num_classes)
print(image_train.shape)
print(train_labels_cat.shape)
print(train_labels_cat[0,:])

(990, 64, 64, 1)
(990, 99)
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]


## build keras model for Images only
### model 1: following the CIFAR 10 example from Keras

In [33]:
img_model = Sequential()

img_model.add(Conv2D(32, (3, 3), padding='same', input_shape=image_train.shape[1:]))
img_model.add(Activation('relu'))
img_model.add(Conv2D(32, (3, 3)))
img_model.add(Activation('relu'))
img_model.add(MaxPooling2D(pool_size=(2, 2)))
img_model.add(Dropout(0.25))

img_model.add(Conv2D(64, (3, 3), padding='same'))
img_model.add(Activation('relu'))
img_model.add(Conv2D(64, (3, 3)))
img_model.add(Activation('relu'))
img_model.add(MaxPooling2D(pool_size=(2, 2)))
img_model.add(Dropout(0.25))

img_model.add(Flatten())
img_model.add(Dense(512))
img_model.add(Activation('relu'))
img_model.add(Dropout(0.5))
img_model.add(Dense(num_classes))
img_model.add(Activation('softmax'))

img_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# output the model summary
img_model.count_params()
img_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
activation_7 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 31, 31, 64)        18496     
__________

In [34]:
# img_history = img_model.fit(image_train_splitted, image_train_label_splitted,
#                             batch_size=16,
#                             epochs=20,
#                             #validation_split=0.1,
#                             validation_data=(image_validation_splitted, image_validation_label_splitted),
#                             shuffle=True)

img_history = img_model.fit_generator(imgen_flow,
                                      epochs=20,
                                      validation_data=(image_validation_splitted, image_validation_label_splitted),
                                      steps_per_epoch=len(image_train)/16)

Epoch 1/20
62/61 [==============================] - 0s - loss: 4.6095 - acc: 0.0060 - val_loss: 5.0361 - val_acc: 0.0000e+00
Epoch 2/20
62/61 [==============================] - 0s - loss: 4.5912 - acc: 0.0131 - val_loss: 15.3930 - val_acc: 0.0202
Epoch 3/20
62/61 [==============================] - 0s - loss: 4.4424 - acc: 0.0267 - val_loss: 15.1321 - val_acc: 0.0404
Epoch 4/20
62/61 [==============================] - 0s - loss: 4.1812 - acc: 0.0478 - val_loss: 14.7992 - val_acc: 0.0808
Epoch 5/20
62/61 [==============================] - 0s - loss: 3.9860 - acc: 0.0650 - val_loss: 14.4929 - val_acc: 0.1010
Epoch 6/20
62/61 [==============================] - 0s - loss: 3.8442 - acc: 0.0917 - val_loss: 13.9912 - val_acc: 0.1111
Epoch 7/20
62/61 [==============================] - 0s - loss: 3.7296 - acc: 0.0962 - val_loss: 13.0388 - val_acc: 0.1818
Epoch 8/20
62/61 [==============================] - 0s - loss: 3.5765 - acc: 0.1214 - val_loss: 13.8455 - val_acc: 0.1313
Epoch 9/20
62/61 [===

In [ ]:
plt.plot(img_history.history['val_acc'])
plt.xlabel('Iterations')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy')
plt.show()

### Model 2: A simpler CNN model using data augmentation method 2

In [13]:
img_model2 = Sequential()

img_model2.add(Conv2D(16, (3, 3), padding='same', input_shape=image_train.shape[1:]))
img_model2.add(Activation('relu'))
img_model2.add(Conv2D(16, (3, 3)))
img_model2.add(Activation('relu'))
img_model2.add(Conv2D(16, (3, 3)))
img_model2.add(Activation('relu'))

img_model2.add(Flatten())
img_model2.add(Dense(128))
img_model2.add(Activation('relu'))
img_model2.add(Dense(64))
img_model2.add(Activation('relu'))
img_model2.add(Dense(num_classes))
img_model2.add(Activation('softmax'))

img_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# output the model summary
# img_model.count_params()
# img_model.summary()

In [21]:
# img_history = img_model2.fit(image_train, train_labels_cat,
#                              batch_size=16,
#                              epochs=10,
#                              validation_split=0.1,
#                              shuffle=True)

image_train_splitted shape (891, 64, 64, 1)
image_validation_splitted shape (99, 64, 64, 1)


In [22]:
img_history = img_model2.fit_generator(imgen_flow,
                                       epochs=20,
                                       validation_data=(image_validation_splitted, image_validation_label_splitted),
                                       steps_per_epoch=len(image_train)/16)

Epoch 1/20
62/61 [==============================] - 1s - loss: 4.6236 - acc: 0.0101 - val_loss: 16.0496 - val_acc: 0.0000e+00
Epoch 2/20
62/61 [==============================] - 1s - loss: 4.5413 - acc: 0.0171 - val_loss: 15.3355 - val_acc: 0.0101
Epoch 3/20
62/61 [==============================] - 1s - loss: 4.2913 - acc: 0.0363 - val_loss: 15.3836 - val_acc: 0.0303
Epoch 4/20
62/61 [==============================] - 1s - loss: 4.1016 - acc: 0.0509 - val_loss: 14.9654 - val_acc: 0.0606
Epoch 5/20
62/61 [==============================] - 1s - loss: 3.8571 - acc: 0.0897 - val_loss: 14.2380 - val_acc: 0.1010
Epoch 6/20
62/61 [==============================] - 1s - loss: 3.6863 - acc: 0.1124 - val_loss: 13.8377 - val_acc: 0.1313
Epoch 7/20
62/61 [==============================] - 1s - loss: 3.5419 - acc: 0.1426 - val_loss: 12.9919 - val_acc: 0.1717
Epoch 8/20
62/61 [==============================] - 1s - loss: 3.4155 - acc: 0.1577 - val_loss: 13.8618 - val_acc: 0.1111
Epoch 9/20
62/61 [==

In [ ]:
plt.plot(img_history.history['val_acc'])
plt.xlabel('Iterations')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy')
plt.show()

## Using Features Only

In [ ]:
# build keras model for Features only
feat_model = Sequential()
feat_model.add(Dense(512, input_dim=192, kernel_initializer='uniform', activation='relu'))
feat_model.add(Dropout(0.3))
feat_model.add(Dense(256, activation='sigmoid'))
feat_model.add(Dropout(0.3))
feat_model.add(Dense(99, activation='softmax'))

feat_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

In [ ]:
# output the model summary
feat_model.count_params()
feat_model.summary()

In [ ]:
feat_history = feat_model.fit(feat_train, train_labels_cat,
                              batch_size=16,
                              epochs=50,
                              validation_split=0.1,
                              shuffle=True)

In [ ]:
plt.plot(feat_history.history['val_acc'])
plt.xlabel('Iterations')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy')
plt.show()

In [ ]:
# do prediction
pred = feat_model.predict_proba(feat_test)
columns = sorted(train.species.unique())
pred = pandas.DataFrame(pred, index=test.id, columns=columns)
output = open('prediction_fully_connected.csv','w')
output.write(pred.to_csv())
output.close()